# Single Cell RNA-seq analysis: Downsizing of Gene by Cell Matrix (For Drop-seq/InDrop/10xGenomics/Seq-well)

## Read in data file and see general expression level distribution

In [50]:
# Import required modules
# Python plotting library
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# Numerical python library (pronounced "num-pie")
import numpy as np
# Dataframes in Python
import pandas as pd
# Statistical plotting library we'll use
import seaborn as sns
# This is necessary to show the plotted figures inside the notebook -- "inline" with the notebook cells
%matplotlib inline
## Add more when seeing new modules in other notebooks!


In [ ]:
# Read in the DGE data sheet
expression = pd.read_table('PathToMatrixFile', 
                               
                                     # Sets the first (Python starts counting from 0 not 1) column as the row names
                                     index_col=0, 

                                     # Tells pandas to decompress the gzipped file if required
                                     compression='gzip')
# Print out the shape and the top 5 rows of the newly assigned dataframe
print(expression.shape)
expression.head()

In [ ]:
# See the raw distributions of data
## e.g. for gene as row name and cell as column name:
## See the distribution of gene expression level within one cell for each of the cells listed?
sns.boxplot(expression)
## Get current figure, then save to file
#fig = plt.gcf()
#fig.savefig(NameOfFigure)


In [ ]:
# Log transform the data if needed
expression_logged = np.log2(expression+1)
expression_logged.head()

In [ ]:
# Plot log transformed data with a designated size
fig, ax=plt.subplots(figsize = (10,5))
sns.boxplot(expression_logged)

# gcf = Get current figure
#fig = plt.gcf()
#fig.savefig(NameOfFigure)

## Determine the cut-off lines for genes and cells
* Genes:
    * Remove genes with no expression in any cells
    * Calculate the number of genes left for each number of transcript need to be detected in at least what number of cells
    _When the computation power is high enough, don't worry about the dimensions (number of genes). Otherwise, may need to cut the number down to 3000_
* Cells:
    * Both low and high expression should be removed
    * Low: smaller than certain number of genes detected (e.g. 1000)
    * High: Look at the distribution of aggregate expression level across all cells. If it shows as bimodal, it is likely that the right hand side peak represents doublets.
    

In [ ]:
# Apply filtering for genes and cells
## Remove no-expression genes
mask0exp = expression.sum(axis=1) > 0
exp0rmd = expression[mask0exp]
print(exp0rmd.shape)
exp0rmd.head()

In [22]:
# Estimate number of gene left for different filtering criteria
# Define a function for calculation
def numGene(mtx,reads_num=5, cell_num=50):
    """
    Estimate number of gene left by applying different filtering criteria. 
    For each pair of expression reads and cell number, calculate the genes that can 
    pass the criteria of have A number of reads detected in at least more than 
    B number of cells.
    Usage: numGene(reads_num, cell_num,mtx)
    Args: 
        reads_num is the upper number of expression level will be tested (count by 1).
        cell_num is the upper number of cell number will be set as threshold (count by 10).
        mtx is the data matrix input gene X cell
    """
    expreads = list(range(0,reads_num))
    cellnum = list(range(10,cell_num + 1,10))
    genenumdict = {}
    for i in expreads:
        z = []
        for j in cellnum:
            gmask = (mtx > i).sum(axis=1) >= j
            genenum = gmask.sum()
            z.append(genenum)
        genenumdict[i] = z
    return genenumdict

In [23]:
# Run numGene() on the matrix
genenumdict = numGene(expression)

In [ ]:
# See how the output gene number estimation looks
genenumdict

In [51]:
# Define a function to plot 3D distribution of the gene number
def plotnumGene(genenumdict, reads_num=5, cell_num=50):
    cellnum = list(range(10,cell_num + 1,10))
    x = np.asarray(cellnum)
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    for i in sorted(genenumdict):
        y = np.ones(x.size)*i
        z = genenumdict[i]
        color_code = (i+0.1)/reads_num
        ax.plot(y,x,z)
    plt.show()
    return

In [ ]:
# Run plotnumGene() to plot
plotnumGene(numGene(expression))